# Web scraping with Python

## HTML basics

### What _is_ a web page, anyway?

Generally, a web page consists of specifically formatted text files. Here are a few common types:

- `.html` (HyperText Markup Language) files give structure to a page
- `.css` (Cascading Style Sheet) files determine how the page looks
- `.js` (JavaScript) files add interactivity

Today, we'll focus on the HTML, which gives structure to the page.

Most HTML elements are represented by a pair of tags -- an opening tag and a closing tag.

A table, for example, starts with `<table>` and ends with `</table>`. The first tag tells the browser: "Hey! I got a table here! Render it as a table." The closing tag (note the forward slash!) tells the browser: "Hey! I'm all done with that table, thanks." Inside the table are nested more HTML tags representing rows (`<tr>`) and cells (`<td>`).


```html
<table id="example-table" class="striped-table" style="width: 95%">
    <tr> <!-- Header -->
        <td>Column A</td>
        <td>Column B</td>
        <td>Column C</td>
    </tr>
    <tr> <!-- Row 1 --->
        <td>Row 1, Column A</td>
        <td>Row 1, Column B</td>
        <td>Row 1, Column C</td>
    </tr>
    <tr> <!-- Row 2 --->
        <td>Row 2, Column A</td>
        <td>Row 2, Column B</td>
        <td>Row 2, Column C</td>
    </tr>
</table>
```

HTML elements can have any number of attributes, such as IDs, which uniquely identify elements --

```html
<table id="example-table">
```

-- classes, which identify a type of element --

```html
<table class="striped-table">
```

-- and styles, which define how specific elements appear --

```html
<table style="width:95%;">
```

-- that will be useful to know about when we're scraping. 

### Inspecting HTML in your browser

You can look at the HTML that makes up a web page by _inspecting the source_ in a web browser. We like Chrome and Firefox for this; today, we'll use Chrome.

#### Inspect element

You can inspect specific elements on the page by right-clicking on the page and selecting "Inspect" or "Inspect Element" from the context menu that pops up. Hover over elements in the "Elements" tab to highlight them on the page. This can be helpful when you're trying to figure how to uniquely identify the element you want to scrape.

#### View page source

To examine all of the source code that makes up a page, you can "view source." In Chrome, right click anywhere in your browser window, then click "View Page Source."

This will open a new tab showing you all of the HTML code that makes up that page. Ignore 99% of it and try to locate the element(s) that you want to target (use `Ctrl+F` on a PC and `⌘+F` to find).

### Practice

Let's give our new skills a whirl. Open up a Chrome browser and navigate to [the first page of Maryland's list of WARN letters](https://www.dllr.state.md.us/employment/warn.shtml). Your goal is to isolate the table element on the page.

There are many ways to grab content from HTML, and every page you scrape data from will require a slightly different trick. At this stage, your job is to find a pattern or identifier in the code for the element you’d like to extract, which we will then give as instructions to our Python code.

Inspect the table element or view the page source. How would you pick the table element out from the all of the elements that make up the page? Is it the only table? If not, does it have any attributes, like a class or ID, that would allow you to target it?

## Web Scraping with Python

The remainder of this notebook demonstrates how you can use the Python programming language to scrape information from a web page. The goal today: Scrape the main table on [the first page of Maryland's list of WARN letters](https://www.dllr.state.md.us/employment/warn.shtml) and, if time, write the data to a CSV that you can open with Excel.

### Import libraries

As we learned in the first section, Python provides some broadly useful objects, like strings and lists, and functions, like `type()` and `print()`, by default. For more specialized operations, like making web requests and parsing HTML, we need to import a few modules, from which we can access helpful objects and functions.

Today, we'll use two third-party Python libraries to help us scrape:
- `requests` is the de facto standard for making HTTP requests, similar to what happens when you type a URL into a browser window and hit enter.
- `bs4`, or BeautifulSoup, is a popular library for parsing HTML into a data structure that Python can work with.

**Note:** These libraries are installed separately from Python on a per-project basis. They're already in your working environment for this tutorial. If you want to revisit this tutorial on your own computer, or if you want to create a scraping project of your own, you can ([read more about IRE's recommendations for setting up Python projects here](https://docs.google.com/document/d/1cYmpfZEZ8r-09Q6Go917cKVcQk_d0P61gm0q8DAdIdg/edit#heading=h.od2v1nkge5t1)).

Use the `import` keyword to import the `requests` and `bs4` modules.

In [2]:
import requests
import bs4
import pandas

Remember, you can use the `dir()` function to inspect the objects and methods included with a module, and the `help()` function to read more about a particular object or method.

In [3]:
dir(requests)

['ConnectTimeout',
 'ConnectionError',
 'DependencyWarning',
 'FileModeWarning',
 'HTTPError',
 'NullHandler',
 'PreparedRequest',
 'ReadTimeout',
 'Request',
 'RequestException',
 'RequestsDependencyWarning',
 'Response',
 'Session',
 'Timeout',
 'TooManyRedirects',
 'URLRequired',
 '__author__',
 '__author_email__',
 '__build__',
 '__builtins__',
 '__cached__',
 '__cake__',
 '__copyright__',
 '__description__',
 '__doc__',
 '__file__',
 '__license__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__title__',
 '__url__',
 '__version__',
 '_check_cryptography',
 '_internal_utils',
 'adapters',
 'api',
 'auth',
 'certs',
 'chardet',
 'check_compatibility',
 'codes',
 'compat',
 'cookies',
 'delete',
 'exceptions',
 'get',
 'head',
 'hooks',
 'logging',
 'models',
 'options',
 'packages',
 'patch',
 'post',
 'put',
 'request',
 'session',
 'sessions',
 'status_codes',
 'structures',
 'urllib3',
 'utils',
 'warnings']

### Request the page

Next, we'll use the `get()` method of the `requests` library (which we just imported) to grab the web page.

Use the `help()` function to learn more about `requests.get`.

**Note**: Some third-party libraries come with more documentation than others, however popular ones (such as `requests`) tend to be just as, if not more, helpful than the standard Python library!

In [9]:
help(requests.get)

Help on function get in module requests.api:

get(url, params=None, **kwargs)
    Sends a GET request.
    
    :param url: URL for the new :class:`Request` object.
    :param params: (optional) Dictionary, list of tuples or bytes to send
        in the query string for the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response



First, we'll define a variable `URL` as a string containing the web address we want to scrape.

In [5]:
URL = 'http://www.dllr.state.md.us/employment/warn.shtml'

Next, we'll use `requests.get` to retrieve the URL. Remember that you can can assign the output of an expression to a variable, so we'll store the response as a new variable. The variable name is arbitrary, but it's a great idea to use something that describes the value it's pointing to.

In [14]:
warn_page = requests.get(URL)
warn_page

<Response [200]>

If you want to make sure that your request was successful, you can check the `status_code` attribute of the Python object that was returned:

In [7]:
warn_page.status_code

200

A `200` code means all is well. `404` means the page wasn't found, etc. [Here's one of our favorite lists of HTTP status codes](https://http.cat/) ([or here, if you prefer dogs](https://httpstatusdogs.com/)).

The object being stored as the `warn_page` variable came back with a lot of potentially useful information we could access. Use the `dir()` method to see all the attributes.

In [10]:
dir(warn_page)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

Today, we're mostly interested in the `.text` attribute -- the HTML that makes up the web page, same as if we'd viewed the page source. Let's take a look:

In [12]:
warn_page.text

'ï»¿<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n    <meta charset="utf-8">\r\n    <title>Work Adjustment and Retraining Notification (WARN) Log Year 2020 - Division of Workforce Development and Adult Learning</title>\r\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0" />\r\n    <meta name="description" content="">\r\n    <meta name="author" content="">\r\n    <!-- Step 1 : Link to MD favicon -->\r\n    <link rel="shortcut icon" href="../egov/img/icons/favicon.ico">\r\n    <!-- Necessary Bootstrap.css for responsive design -->\r\n    <link href="../egov/frameworks/bootstrap/css/bootstrap.min.css" rel="stylesheet">\r\n    <link href="../egov/css/agency-styles.css" rel="stylesheet">\r\n    <link href="../egov/css/dllrstyle.css" rel="stylesheet" type="text/css">\r\n    <link href="../egov/css/agency-header.css" rel="stylesheet">\r\n    <!--[if ie 8]>\r\n        <link id="css_IE8" href="../egov/css/ie/ie8.css" rel="stylesheet" media="screen" 

### ✍️ Try it yourself

Use the code blocks below to experiment with requesting web pages and checking out the HTML that gets returned.

Some ideas to get you started:
- `'http://ire.org'`
- `'https://web.archive.org/web/20031202214318/http://www.tdcj.state.tx.us:80/stat/finalmeals.htm'`
- `'https://www.nrc.gov/reactors/operating/list-power-reactor-units.html'`

In [16]:
url2 = 'https://web.archive.org/web/20031202214318/http://www.tdcj.state.tx.us:80/stat/finalmeals.htm'

In [17]:
response = requests.get(url2)

### Turn your HTML into soup

The HTML in the `.text` attribute of the request object is just a string -- a big ol' chunk of text.

Before we start targeting and extracting pieces of data in the HTML, we need to turn that chunk of text into a data structure that Python can work with. That's where the [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) (`bs4`) library comes in.

We'll create a new instance of a `BeautifulSoup` object, which lives under the top-level `bs4` library that we imported earlier. We need to give it two things:
- The HTML we'd like to parse -- `warn_page.text`
- A string with the name of the type of parser to use -- `html.parser` is the default and usually fine, but [there are other options](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser)

We'll save the parsed HTML as a new variable, `soup`.

In [19]:
soup = bs4.BeautifulSoup(warn_page.text, 'html.parser')

In [22]:
type(warn_page.text)

str

Nothing happened, which is good! You can take a look at what `soup` is, but it looks pretty much like `warn_page.text`, except a bit easier to read:

In [23]:
type(soup)

bs4.BeautifulSoup

If you want to be sure, you can use the Python function `type()` to check what sort of object you're dealing with:

In [9]:
# the `str` type means a string, or text
type(warn_page.text)

str

In [10]:
# the `bs4.BeautifulSoup` type means we successfully created the object
type(soup)

bs4.BeautifulSoup

### ✍️ Try it yourself

Use the code blocks below to experiment fetching HTML and turning it into soup (if you fetched some pages earlier and saved them as variables, that'd be a good start).

### Target and extract the data

Now that we have BeautifulSoup object loaded up, we can go hunting for the specific HTML elements that contain the data we need. Our general strategy:
1. Find the main table with the data we want to grab
2. Get a list of rows (the `tr` element, which stands for "table row") in that table
3. Use a Python `for loop` to go through each table row and find the data inside it (`td`, or "table data")

To accomplish this, we'll use two `bs4` methods:
- [`find()`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find), which returns the first element that matches whatever criteria you hand it
- [`find_all()`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all), which returns a _list_ of elements that match the criteria. ([Here's how Python lists work](Python%20syntax%20cheat%20sheet.ipynb#Lists).)

#### Find the table

To start with, we need to find the table. There are several ways to accomplish this, but because this is the only table on the page (view source and `Ctrl+F` to search for `<table` to confirm), we can simply say, "Look through the `soup` object and find the table tag."

Translated, the code is: `soup.find('table')`. While we're at it, save the results of that search to a new variable, `table`.

Run these cells:

In [24]:
table = soup.find('table')

In [26]:
type(table)

bs4.element.Tag

#### Find the rows in the table

Next, use the `find_all()` method to drill down and get a list of rows in the table:

In [27]:
rows = table.find_all('tr')

In [32]:
# rows

To see how many items are in this list -- in other words, how many rows are in the table -- you can use the `len()` function:

In [33]:
len(rows)

79

#### Loop through the rows and extract the data

Next, we can use a [`for` loop](Python%20syntax%20cheat%20sheet.ipynb#for-loops) to go through the list of rows and start grabbing data from each one.

Quick refresher on _for loop_ syntax: Start with the word `for` (lowercase), then a variable name to stand in for each item in the list that you're looping over, then the word `in` (lowercase), then the name of the list holding the items (`rows`, in our case), then a colon, then an indented block of code describing what we're doing to each item in the list.

Each piece of data in the row will be stored in a `td` tag, which stands for "table data." So inside the loop -- in the indented block -- we'll use the `find_all()` method to get a list of every `td` tag inside the row. And from there, we can access the content inside each tag.

Our goal is to end up with a _list_ of data for each row that we will eventually write out to a file. Typically you'd probably do the work of looping and inspecting the results, step by step, in one code cell. But to show the thinking of how you might approach this (and to practice the syntax), we'll start by just printing out each row and then build from there. (`print('='*80)` will print a line of 80 equals signs -- a way to help us see exactly what we're working with in each row.)

In [51]:
for row in rows:
    #print(row)
    #print('='*80)  # prints a divider line for easier visibility
    pass

Notice that the first item that prints is the header row with the column labels. You are free to keep these headers if you want, but I typically skip that row and define my own list of column names.

(Another thing to consider: On better-constructed web pages, the cells in the header row will be represented by `th` ("table header") tags, not `td` ("table data") tags. The next step in our `for` loop is, "Find all of the `td` tags in this row," so that would be something you would need to deal with.)

We can skip the first row by using _list slicing_: adding square brackets after the name of the list with some instructions about which items in the list we want to select.

Here, the syntax would be: `rows[1:]`, which means, take everything in the `rows` list starting with the item in position 1 (the second item) to the end of the list. Like many programming languages, Python starts counting at 0, so the result will leave off the first item in the list -- i.e. the item in position 0, i.e. the headers.

In [37]:
for row in rows[1:]:
    #print(row)
    #print('='*80)
    pass

Now we're cooking with gas! Now that we've isolated the rows we want, let's start pulling the data out of each row. Start by using `find_all()` to grab a list of `td` tags:

In [39]:
for row in rows[1:]:
    cells = row.find_all('td')
    #print(cells)
    #print('='*80)

Now we have, for each row, a _list_ of `td` tags. Next step is to look at the table and start grabbing specific values based on their position in the list and assigning them to human-readable variable names.

Quick refresher on list syntax: To access a specific item in a list, use square brackets `[]` and the index number of the item you'd like to access. For instance, to get the first cell in the row -- the date that each WARN report was issued -- use `[0]`.

In [41]:
for row in rows[1:]:
    cells = row.find_all('td')
    warn_date = cells[0]
    print(warn_date)
    print(type(warn_date))
    print('='*80)

<td>1/10/2020</td>
<class 'bs4.element.Tag'>
<td>1/15/2020</td>
<class 'bs4.element.Tag'>
<td>1/29/2020</td>
<class 'bs4.element.Tag'>
<td>1/29/2020</td>
<class 'bs4.element.Tag'>
<td>2/11/2020</td>
<class 'bs4.element.Tag'>
<td>2/18/2020</td>
<class 'bs4.element.Tag'>
<td>2/18/2020</td>
<class 'bs4.element.Tag'>
<td>3/5/2020</td>
<class 'bs4.element.Tag'>
<td>3/5/2020</td>
<class 'bs4.element.Tag'>
<td>3/5/2020</td>
<class 'bs4.element.Tag'>
<td>3/9/2020</td>
<class 'bs4.element.Tag'>
<td>3/9/2020</td>
<class 'bs4.element.Tag'>
<td>3/9/2020</td>
<class 'bs4.element.Tag'>
<td>3/9/2020</td>
<class 'bs4.element.Tag'>
<td>3/9/2020</td>
<class 'bs4.element.Tag'>
<td>3/9/2020</td>
<class 'bs4.element.Tag'>
<td>3/9/2020</td>
<class 'bs4.element.Tag'>
<td>3/18/2020</td>
<class 'bs4.element.Tag'>
<td>3/18/2020</td>
<class 'bs4.element.Tag'>
<td>3/19/2020</td>
<class 'bs4.element.Tag'>
<td>3/19/2020</td>
<class 'bs4.element.Tag'>
<td>3/19/2020</td>
<class 'bs4.element.Tag'>
<td>3/19/2020</td>
<

This is returning the entire `Tag` object -- we just want the contents inside it. You can access the `.text` attribute of the tag to get the text inside:

In [42]:
for row in rows[1:]:
    cells = row.find_all('td')
    warn_date = cells[0].text    
    print(warn_date)

1/10/2020
1/15/2020
1/29/2020
1/29/2020
2/11/2020
2/18/2020
2/18/2020
3/5/2020
3/5/2020
3/5/2020
3/9/2020
3/9/2020
3/9/2020
3/9/2020
3/9/2020
3/9/2020
3/9/2020
3/18/2020
3/18/2020
3/19/2020
3/19/2020
3/19/2020
3/19/2020
3/23/2020
3/25/2020
3/25/2020
3/26/2020
3/26/2020
3/26/2020
3/30/2020
3/30/2020
3/31/2020
4/1/2020
4/1/2020
3/30/2020
4/1/2020
4/1/2020
4/1/2020
4/1/2020
4/1/2020
4/1/2020
4/1/2020
4/1/2020
4/1/2020
4/2/2020
4/3/2020
4/3/2020
4/3/2020
4/6/2020
4/6/2020
4/6/2020
4/6/2020
4/6/2020
4/6/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/8/2020
4/9/2020
4/6/2020
4/6/2020
4/6/2020
4/6/2020
4/13/2020


In the next cell (`[1]`), the `.text` attribute will give you the NAICS code. In the third cell (`[2]`) you'll get the name of the business. Etc.

It's also generally good practice to trim off external whitespace for each value, and you can use the Python built-in string method `strip()` to accomplish this as you march across the row.

Which gets us this far:

In [43]:
for row in rows[1:]:
    cells = row.find_all('td')
    warn_date = cells[0].text.strip()
    naics_code = cells[1].text.strip()
    biz = cells[2].text.strip()
    print(warn_date, naics_code, biz)

1/10/2020 452210 Macy's
1/15/2020 493110 IAS Logistics DFW, LLC                           d/b/a                     Pinnacle Logistics
1/29/2020 493110 Collington Distribution Center
1/29/2020 531311 Oakwood Worldwide (US) LP
2/11/2020 492110 Fedex Ground Package System, Inc.
2/18/2020 811111 Transform SR LLC-06063                  (Sears Auto Center)
2/18/2020 452210 Transform SR LLC-01074                  (Sears Department Store)
3/5/2020 624190 Salvation Army Adult Rehabilitation Centers
3/5/2020 624190 Salvation Army Adult Rehabilitation Centers
3/5/2020 624190 Salvation Army Adult Rehabilitation Centers
3/9/2020 442110 Wolf Furniture
3/9/2020 442110 Wolf Furniture
3/9/2020 442110 Wolf Furniture
3/9/2020 442110 Wolf Furniture
3/9/2020 442110 Wolf Furniture
3/9/2020 442110 Wolf Furniture
3/9/2020 442110 Wolf Furniture
3/18/2020 611620 Silver Spring Gymnastics & Fitness
3/18/2020 812930 SP Plus Corporation
3/19/2020 722515 Artifact Coffee
3/19/2020 722511 Foodshed, LLC
3/19/2020 7225

### ✍️ Try it yourself

Now that you've gotten this far, see if you can isolate the other pieces of data in each row.

In [59]:
results = []
for row in rows[1:]:
    cells = row.find_all('td')
    warn_date = cells[0].text.strip()
    naics_code = cells[1].text.strip()
    biz = cells[2].text.strip()
    
    # address
    address = cells[3].text.strip()
    # wia_code
    wia_code = cells[4].text.strip()
    # total_employees
    total_employees = cells[5].text.strip()
    # effective_date
    effective_date = cells[6].text.strip()
    # type_code
    type_code = cells[7].text.strip()
    # print()
    obj = {'warn_date': warn_date,
           'naics_code': naics_code,
           'biz': biz,
           'address': address,
           'wia_code': wia_code,
           'total_employees': total_employees,
           'effective_date': effective_date,
           'type_code': type_code
          }
    results.append(obj)

In [52]:
import pandas as pd

In [87]:
df = pd.DataFrame(results)
df

,warn_date,naics_code,biz,address,wia_code,total_employees,effective_date,type_code
0,1/10/2020,452210,Macy's,"2304 N. Salisbury Blvd Salisbury, MD 21801",5,80,3/30/2020,1
1,1/15/2020,493110,"IAS Logistics DFW, LLC ...",BWI Airport 7050 Friendship Ro...,1,"1,609",4/14/2020,2
2,1/29/2020,493110,Collington Distribution Center,"16020 Leeland Road Upper Marlboro, MD 20774",8,520,8/1/2020,1
3,1/29/2020,531311,Oakwood Worldwide (US) LP,888 Airport Park Road Suite A ...,1,2,3/30/2020,2
4,2/11/2020,492110,"Fedex Ground Package System, Inc.","4801 Hollins Ferry Road Halethorpe, MD 21227",2,130,4/18/2020,1
...,...,...,...,...,...,...,...,...
73,4/6/2020,448310,The Silver Heron,The Mall at Columbia 10300 Little Patu...,6,4,4/4/2020,2
74,4/6/2020,448310,Pandora,St. Charles Towne Center 11110 Mall...,9,10,4/4/2020,2
75,4/6/2020,448310,Pandora,The Mall at Columbia 10300 Little Patu...,6,17,4/4/2020,2
76,4/6/2020,448310,Pandora,The Gallery at Harborplace 300 ...,3,7,4/4/2020,2


In [88]:
df['total_employees'] = pd.to_numeric(df['total_employees'].str.replace(',', ''))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   warn_date        78 non-null     object
 1   naics_code       78 non-null     object
 2   biz              78 non-null     object
 3   address          78 non-null     object
 4   wia_code         78 non-null     object
 5   total_employees  78 non-null     int64 
 6   effective_date   78 non-null     object
 7   type_code        78 non-null     object
dtypes: int64(1), object(7)
memory usage: 5.0+ KB


In [90]:
df.sort_values('total_employees', ascending=False)

,warn_date,naics_code,biz,address,wia_code,total_employees,effective_date,type_code
1,1/15/2020,493110,"IAS Logistics DFW, LLC ...",BWI Airport 7050 Friendship Ro...,1,1609,4/14/2020,2
2,1/29/2020,493110,Collington Distribution Center,"16020 Leeland Road Upper Marlboro, MD 20774",8,520,8/1/2020,1
44,4/2/2020,721110,Four Seasons Hotel Baltimore,"200 International Drive Baltimore, MD 21202",3,169,3/23/2020,2
46,4/3/2020,493110,Under Armour,"7030 Tradepoint Avenue Edgemere, Md ...",2,154,4/12/2020,1
22,3/19/2020,721110,Hotel at UMCP,7777 Baltimore Avenue College ...,8,150,3/19/2020,2
...,...,...,...,...,...,...,...,...
20,3/19/2020,722511,"Foodshed, LLC",2031 Clipper Park Road ...,3,4,3/16/2020,2
3,1/29/2020,531311,Oakwood Worldwide (US) LP,888 Airport Park Road Suite A ...,1,2,3/30/2020,2
50,4/6/2020,561720,"Meta Coastal, LLC",350 N. Washington Street ...,11,2,3/20/2020,2
56,4/8/2020,448120,RTW Retailwinds ...,"3317 Rosary Court Abingdon,...",10,1,3/29/2020,2


In [57]:
df.to_csv("output.csv", index=False)

### Write the results to file

Now that we've targeted our lists of data for each row, we can use Python's built-in [`csv`](https://docs.python.org/3/library/csv.html) module to write each list to a CSV file.

First, import the csv module.

In [ ]:
import csv

Now define a list of headers to match the data (each column header will be a string) -- run this cell:

In [ ]:
HEADERS = ['warn_date', 'naics_code', 'biz', 'address', 'wia_code',
           'total_employees', 'effective_date', 'type_code']

Now, using something called a `with` block, open a new CSV file to write to and write some code to do the following things:
- Create a `csv.writer` object
- Write out the list of headers using the `writerow()` method of the `csv.writer` object
- Drop in the `for` loop you just wrote and, instead of just printing the contents of each cell, create a list of items and use the `writerow()` method of the `csv.writer` object to write your list of data to file

In [ ]:
# create a file called 'warn-data.csv' in write ('w') mode
# specify that newlines are terminated by an empty string (this deals with a PC-specific problem)
# and use the `as` keyword to name the open file handler (the variable name `outfile` is arbitrary)
with open('warn-data.csv', 'w', newline='') as outfile:
    # go to the csv module we imported and make a new .writer object attached to the open file
    # and save it to a variable
    writer = csv.writer(outfile)

    # write out the list of headers
    writer.writerow(headers)
    
    # paste in the for loop you wrote earlier here -- watch the indentation!
    # it should be at this indentation level =>
    # for row in rows[1:]:
    #     cells = row.find_all('td')
    #     etc. ...
    # but at the end, instead of `print(warn_date, naics_code, ...etc.)`
    # make it something like
    # data_out = [warn_date, naics_code, ...etc.]
    # `writer.writerow(data_out)`

If you look in the folder, you should see a new file: `warn-data.csv`. Hooray!

🎉 🎉 🎉

### ✍️ Try it yourself

Putting it all together:
- Find a website you'd like to scrape
- Use `requests` to fetch the HTML
- Use `bs4` to parse the HTML and isolate the data you're interested in
- Use `csv` to write the data to file

### Extra credit problems

1. **Remove internal whitespace:** Looking over the data, you probably noticed that some of the values have some unnecessary internal whitespace, which you could fix before you wrote each row to file. Python does not have a built-in string method to remove internal whitespace, unfortunately, but [Googling around](https://www.google.com/search?q=python+remove+internal+whitespace) will yield you a common strategy: Using the `split()` method to separate individual words in the string, then `join()`ing the resulting list on a single space. As an example:

```python
my_text = 'hello     world      how are      you?'

# split() will turn this into a list of words
my_text_words = my_text.split()
# ['hello', 'world', 'how', 'are', 'you?']

# join on a single space
my_text_clean = ' '.join(my_text_words)
print(my_text_clean)
# prints 'hello world how are you?'

# or, as a one-liner
my_text_clean = ' '.join(my_text.split())
```

2. **Fetch multiple years:** The table we scraped has WARN notices for the current year, but the agency also maintains pages with WARN notices for previous years -- there's a list of them in a section [toward the bottom of the page](https://www.dllr.state.md.us/employment/warn.shtml). See if you can figure out how to loop over multiple pages and scrape the contents of each into a single CSV.


3. **Build a lookup table:** Each numeric code in the "WIA Code" column correspondes to a local area. See if you can figure out how to create a lookup dictionary that maps the numbers to their locations, then as you're looping over the data table, replace the numeric value in that column with the name of the local area instead. Here's a hint:

```python
    lookup_dict = {
        '1': 'hello',
        '2': 'world'
    }

    print(lookup_dict.get('1'))
    # prints 'hello'

    print(lookup_dict.get('3'))
    # prints None

```


4. **Fix encoding errors:** You might have noticed a few encoding problems -- e.g., `Nestlé` is being renedered as `NestlÃ©`. This is due to an encoding problem -- the `warn_page.text` is not encoded as `utf-8`. Using `decode()` and `encode()`, see if you can fix this. (Hint! It looks like the state of Maryland is a big fan of `latin-1`.)